In [1]:
import pandas as pd
import numpy as np
import nltk
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem.porter import PorterStemmer

import re
import codecs
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
n_jobs=-1

In [2]:
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    return df

In [3]:
# X_train, X_val, y_train, y_val = train_test_split(df.company_description, 
#                                                   df.category_id, 
#                                                   test_size=0.15, 
#                                                   random_state=42, 
#                                                   stratify=df.category_id)

In [4]:
X_train=pd.read_csv('../data/X_train.csv')
X_test=pd.read_csv('../data/X_test.csv')
y_train=pd.read_csv('../data/y_train.csv')
y_test=pd.read_csv('../data/y_test.csv')

In [5]:
X_train_clean = clean_text(X_train, 'company_description', 'text_clean')
X_train['company_description'] = X_train_clean['text_clean']
del X_train_clean

X_test_clean = clean_text(X_test, 'company_description', 'text_clean')
X_test['company_description'] = X_test_clean['text_clean']
del X_test_clean

In [6]:
X_train=X_train.drop(['company_address_encoded','company_name','_id','text_clean'],axis=1)
X_test=X_test.drop(['company_address_encoded','company_name','_id','text_clean'],axis=1)

In [7]:
X_train = X_train.drop(['company_sector_encoded','company_description'],axis=1)
X_test = X_test.drop(['company_sector_encoded','company_description'],axis=1)

In [8]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

X_train = MinMaxScaler().fit_transform(X_train)
X_test = MinMaxScaler().fit_transform(X_test)

In [9]:
# # roc curve and auc
# from sklearn.datasets import make_classification
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import roc_curve
# from sklearn.metrics import roc_auc_score
# from matplotlib import pyplot
# # generate 2 class dataset
# X, y = make_classification(n_samples=1000, n_classes=2, random_state=1)
# # split into train/test sets
# trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)
# # generate a no skill prediction (majority class)
# ns_probs = [0 for _ in range(len(testy))]
# # fit a model
# model = LogisticRegression(solver='lbfgs')
# model.fit(trainX, trainy)
# # predict probabilities
# lr_probs = model.predict_proba(testX)
# # keep probabilities for the positive outcome only
# lr_probs = lr_probs[:, 1]
# # calculate scores
# ns_auc = roc_auc_score(testy, ns_probs)
# lr_auc = roc_auc_score(testy, lr_probs)
# # summarize scores
# print('No Skill: ROC AUC=%.3f' % (ns_auc))
# print('Logistic: ROC AUC=%.3f' % (lr_auc))
# # calculate roc curves
# ns_fpr, ns_tpr, _ = roc_curve(testy, ns_probs)
# lr_fpr, lr_tpr, _ = roc_curve(testy, lr_probs)
# # plot the roc curve for the model
# pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
# pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# # axis labels
# pyplot.xlabel('False Positive Rate')
# pyplot.ylabel('True Positive Rate')
# # show the legend
# pyplot.legend()
# # show the plot
# pyplot.show()

In [10]:
# from sentence_transformers import SentenceTransformer
# bert = SentenceTransformer('stsb-roberta-large') #RoBERTa, an optimized version of BERT by Facebook.

In [11]:
# # vectorize the data
# X_train_vec = pd.DataFrame(np.vstack(X_train.company_description.apply(bert.encode)))
# X_test_vec = pd.DataFrame(np.vstack(X_test.company_description.apply(bert.encode)))

In [12]:
# X_train_vec.to_csv('../data/bert_encoded_train.csv',index=False)
# X_test_vec.to_csv('../data/bert_encoded_test.csv',index=False)
# y_train.to_csv('../data/for_bert_emb_Y_train.csv',index=False)
# y_test.to_csv('../data/for_bert_emb_Y_test.csv',index=False)

In [13]:
#reading bert encoded company description
X_train_vec=pd.read_csv('../data/bert_encoded_train.csv')
X_test_vec=pd.read_csv('../data/bert_encoded_test.csv')
y_train=pd.read_csv('../data/for_bert_emb_Y_train.csv')
y_test=pd.read_csv('../data/for_bert_emb_Y_test.csv')

In [14]:
X_train_vec = np.concatenate((X_train_vec, X_train),axis = 1)
X_test_vec = np.concatenate((X_test_vec, X_test),axis = 1)

In [15]:
# scalar = MinMaxScaler()
# # fitting
# scalar.fit(X_train)
# scaled_data = scalar.transform(X_train)

# # Importing PCA
# from sklearn.decomposition import PCA

# pca = PCA(n_components = 10)
# pca.fit(scaled_data)
# x_pca_train = pca.transform(scaled_data)

# x_pca_train.shape

# scalar.fit(X_test)
# scaled_data_x_test = scalar.transform(X_test)

# pca = PCA(n_components = 10)
# pca.fit(scaled_data_x_test)
# x_pca_test = pca.transform(scaled_data_x_test)

# x_pca_test.shape

In [16]:
# scalar.fit(X_train_vec)
# scaled_data_bert_train = scalar.transform(X_train_vec)

# pca = PCA(n_components = 100)
# pca.fit(scaled_data_bert_train)
# x_pca_bert_train = pca.transform(scaled_data_bert_train)

# x_pca_bert_train.shape


# scalar.fit(X_test_vec)
# scaled_data_bert_test = scalar.transform(X_test_vec)

# pca = PCA(n_components = 100)
# pca.fit(scaled_data_bert_test)
# x_pca_bert_test = pca.transform(scaled_data_bert_test)

# x_pca_bert_test.shape

In [17]:
y_train=y_train.values.ravel()
y_test=y_test.values.ravel()

In [18]:
'''RANDOM FOREST '''
model = RandomForestClassifier(n_estimators=500, n_jobs=-1,verbose=1)
model.fit(X_train_vec, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 438 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  6.3min finished


RandomForestClassifier(n_estimators=500, n_jobs=-1, verbose=1)

In [19]:
model.score(X_test_vec, y_test)

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    1.9s
[Parallel(n_jobs=6)]: Done 500 out of 500 | elapsed:    2.2s finished


0.3081901717938474

In [20]:
y_pred=model.predict(X_test_vec)

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    1.9s
[Parallel(n_jobs=6)]: Done 500 out of 500 | elapsed:    2.1s finished


In [20]:
# X_train_vec = np.concatenate((x_pca_bert_train, x_pca_train),axis = 1)
# X_test_vec = np.concatenate((x_pca_bert_test, x_pca_test),axis = 1)

In [21]:
# #BERT doesn't have feature names, 
# #use your classifier
# '''RANDOM FOREST '''
# model = RandomForestClassifier(n_estimators=200, n_jobs=8,verbose=1)
# model.fit(X_train_vec, y_train)

# # model.score(X_test_vec, y_test)

In [21]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.21      0.78      0.33      1523
           1       0.86      0.41      0.55       135
           2       0.42      0.65      0.51       720
           3       0.57      0.11      0.19       107
           4       0.00      0.00      0.00        85
           5       0.32      0.56      0.41       131
           6       0.42      0.11      0.17       157
           7       0.00      0.00      0.00        87
           8       0.00      0.00      0.00        26
           9       0.25      0.28      0.27      1036
          10       0.43      0.62      0.51       224
          11       0.40      0.71      0.51       486
          12       1.00      0.04      0.08        48
          13       0.00      0.00      0.00         5
          14       0.21      0.32      0.25       789
          15       0.00      0.00      0.00        56
          16       0.43      0.06      0.10        53
          17       0.00    

/home/slintel/bert/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/slintel/bert/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/slintel/bert/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
LR_model = LogisticRegression(n_jobs=-1,verbose=True)
LR_model.fit(X_train_vec, y_train)
y_pred = LR_model.predict(X_test_vec)
acc_score = LR_model.score(X_test_vec, y_test)
print(acc_score)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.5min finished


0.4619256891729924


In [24]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.49      0.47      1523
           1       0.80      0.76      0.78       135
           2       0.55      0.67      0.61       720
           3       0.38      0.49      0.43       107
           4       0.24      0.16      0.20        85
           5       0.42      0.48      0.45       131
           6       0.41      0.43      0.42       157
           7       0.29      0.22      0.25        87
           8       0.23      0.12      0.15        26
           9       0.41      0.41      0.41      1036
          10       0.48      0.61      0.54       224
          11       0.57      0.64      0.60       486
          12       0.58      0.54      0.56        48
          13       0.17      0.20      0.18         5
          14       0.34      0.32      0.33       789
          15       0.18      0.12      0.15        56
          16       0.45      0.49      0.47        53
          17       0.49    

/home/slintel/bert/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/slintel/bert/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/slintel/bert/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
from sklearn.svm import SVC

clf = SVC(kernel='linear', C = 1.0)
clf.fit(X_train_vec,y_train)
y_pred = clf.predict(X_test_vec)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.43531761885737114


In [ ]:
'''SVM'''
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(SVC(kernel='linear', probability=True,verbose=True), n_jobs=-1)
# model = LinearSVC(verbose=1,n_jobs=-1)
clf.fit(X_train_vec, y_train)

In [ ]:
clf.score(X_test_vec, y_test)

In [ ]:
y_pred = clf.predict(X_test_vec)
print(metrics.classification_report(y_test, y_pred)

In [ ]:
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=df.company_industry.values, yticklabels=df.company_industry.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
y_train=y_train.values.ravel()
y_test=y_test.values.ravel()

In [ ]:
import time
import numpy as np
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

X=X_train
y=y_train

start = time.time()
clf = OneVsRestClassifier(SVC(kernel='linear', probability=True, class_weight='auto'.n_jobs=-1,verbose=1))
clf.fit(X, y)
end = time.time()
print "Single SVC", end - start, clf.score(X,y)
proba = clf.predict_proba(X)

n_estimators = 10
start = time.time()
clf = OneVsRestClassifier(BaggingClassifier(SVC(kernel='linear', probability=True, class_weight='auto',n_jobs=-1,verbose=1), max_samples=1.0 / n_estimators, n_estimators=n_estimators))
clf.fit(X, y)
end = time.time()
print "Bagging SVC", end - start, clf.score(X,y)
proba = clf.predict_proba(X)

start = time.time()
clf = RandomForestClassifier(min_samples_leaf=20,n_jobs=-1,verbose=True)
clf.fit(X, y)
end = time.time()
print "Random Forest", end - start, clf.score(X,y)
proba = clf.predict_proba(X)

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt

disp = plot_precision_recall_curve(classifier, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))